In [46]:
# 常用
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import os
# 資料預處理用
import math
import pywt
# 讓每一種標籤的資料都平均一點
import random
# 機器學習相關
import torch
import torch.utils.data as Data
import torch.nn as nn
import torchvision
import torch.nn.functional as F
# 正規化
from sklearn import preprocessing
# 訓練結果報告分析
from sklearn import metrics

In [47]:
# 讀取資料的模組
def data_reading(path):
    print('【讀取檔案】')
    data = []
    for file in os.listdir(path):
        data.append(sio.loadmat(path + '/' + file))
        plt.close()
    print('檔案讀取完畢！共讀取了', len(data), '個檔案的資料\n')
    return data
# 讀取受試者資料
def subject_choose(data, subject_ID_list):
    print('【讀取受試者資料】')
    # subject_emg[受試者編號][emg編號][訊號]
    # subject_emg[受試者編號][狀態]
    subject_emg = []
    subject_restimulus = []
    # 選擇受試者
    for subject_ID in subject_ID_list:
        print('受試者', subject_ID, '資料讀取')
        sub_data = list(filter(lambda x: np.squeeze(x['subject']) == subject_ID, data))[0]
        subject_emg.append(sub_data['emg'].T)
        subject_restimulus.append(sub_data['restimulus'])
    print('受試者資料讀取完畢！\n')
    return subject_emg, subject_restimulus

In [48]:
# 資料預處理

def wavelet_with_energy_sum(sliding_window_size, sliding_duration, wavelet, mode, maxlevel):
    print('【預處理-小波包能量】')
    x_data = []
    y_data = []
    count = 0
    for emg, restimulus in zip(subject_emg, subject_restimulus):
        count += 1
        print('第' + str(count) + '位受試者資料預處理')
        total_len = len(restimulus)
        # math.ceil -> 無條件進位
        sliding_times = math.ceil((total_len - sliding_window_size) / sliding_duration) + 1

        # 資料分割 + 特徵提取
        window_begin = 0
        for i in range(sliding_times):
        #   特徵提取
            feature_matrix = []
            for e in emg:
                emg_segment = e[window_begin:window_begin+sliding_window_size]
            #   使用多階小波包轉換
            #   小波包基底: db5
            #   層數: 4
                wp = pywt.WaveletPacket(data=emg_segment, wavelet=wavelet, mode=mode, maxlevel=maxlevel)
            #   對第四層每一節點做能量值計算
                wavelet_energy = []
                for j in [node.path for node in wp.get_level(wp.maxlevel, 'natural')]:
                    wavelet_energy.append(np.sum( (np.array(wp[j].data)) ** 2 ))
                feature_matrix.append(wavelet_energy)
            x_data.append(feature_matrix)
        #   標標籤
            restimulus_segment = restimulus[window_begin:window_begin+sliding_window_size]
        #   np.sqeeze()把矩陣內的單維向量的框框消掉
            counts = np.bincount(np.squeeze(restimulus_segment))
            #返回眾數(注意:此方法只有非負數列才可使用)
            label_action_ID = np.argmax(counts)
            y_data.append(label_action_ID)
            window_begin = window_begin + sliding_duration
    print('資料預處理完畢！共', len(x_data), '筆資料\n')
    print('資料標籤數量分布：', np.bincount(np.squeeze(y_data)))
#     讓 label 0 的資料減少一點，使資料分布平均
    x_filter_data = []
    y_filter_data = []
    for i in range(len(x_data)):
        if y_data[i] == 0:
            if random.randint(1, round(np.bincount(np.squeeze(y_data))[0]/np.bincount(np.squeeze(y_data))[1]) ) == 1:
                x_filter_data.append(x_data[i])
                y_filter_data.append(y_data[i])
        else:
            x_filter_data.append(x_data[i])
            y_filter_data.append(y_data[i])
    x_data = x_filter_data
    y_data = y_filter_data
    del x_filter_data
    del y_filter_data
    print('\n資料篩選完成')
    print('資料數量： x -> ', len(x_data), ', y ->', len(y_data))
    print('資料標籤分佈：', np.bincount(np.squeeze(y_data)),'\n')
    # 正規化
    x_data = list(preprocessing.scale(np.array(x_data).reshape(-1)).reshape(-1, 10, 16))
    for i in range(len(x_data)):
        x_data[i] = [x_data[i]]
    return x_data, y_data

def wavelet_parameters_only(sliding_window_size, sliding_duration, wavelet, mode, maxlevel):
    print('【預處理-小波包係數】')
    x_data = []
    y_data = []
    count = 0
    for emg, restimulus in zip(subject_emg, subject_restimulus):
        count += 1
        print('第' + str(count) + '位受試者資料預處理')
        total_len = len(restimulus)
        # math.ceil -> 無條件進位
        sliding_times = math.ceil((total_len - sliding_window_size) / sliding_duration) + 1

        # 資料分割 + 特徵提取
        window_begin = 0
        for i in range(sliding_times):
        #   特徵提取
            feature_matrix = []
            for e in emg:
                emg_segment = e[window_begin:window_begin+sliding_window_size]
            #   使用多階小波包轉換
            #   小波包基底: db5
            #   層數: 4
                wp = pywt.WaveletPacket(data=emg_segment, wavelet=wavelet, mode=mode, maxlevel=maxlevel)
                wavelet_parameter = []
                for j in [node.path for node in wp.get_level(wp.maxlevel, 'natural')]:
                    wavelet_parameter.append(wp[j].data)
                feature_matrix.append(wavelet_parameter)
            x_data.append(feature_matrix)
        #   標標籤
            restimulus_segment = restimulus[window_begin:window_begin+sliding_window_size]
        #   np.sqeeze()把矩陣內的單維向量的框框消掉
            counts = np.bincount(np.squeeze(restimulus_segment))
            #返回眾數(注意:此方法只有非負數列才可使用)
            label_action_ID = np.argmax(counts)
            y_data.append(label_action_ID)
            window_begin = window_begin + sliding_duration
    print('資料預處理完畢！共', len(x_data), '筆資料\n')
    print('資料標籤數量分布：', np.bincount(np.squeeze(y_data)))
#     讓 label 0 的資料減少一點，使資料分布平均
    x_filter_data = []
    y_filter_data = []
    for i in range(len(x_data)):
        if y_data[i] == 0:
            if random.randint(1, round(np.bincount(np.squeeze(y_data))[0]/np.bincount(np.squeeze(y_data))[1]) ) == 1:
                x_filter_data.append(x_data[i])
                y_filter_data.append(y_data[i])
        else:
            x_filter_data.append(x_data[i])
            y_filter_data.append(y_data[i])
    x_data = x_filter_data
    y_data = y_filter_data
    del x_filter_data
    del y_filter_data
    print('\n資料篩選完成')
    print('資料數量： x -> ', len(x_data), ', y ->', len(y_data))
    print('資料標籤分佈：', np.bincount(np.squeeze(y_data)),'\n')
    # 正規化
    print(np.array(x_data).shape)
    x_data = list(preprocessing.scale(np.array(x_data).reshape(-1)).reshape(-1, 10, 16, 20))
    return x_data, y_data

In [53]:
# 機器學習模組

# 資料集設定
def data_setting(BATCH_SIZE):
    print('【資料集設定】')
    # 先转换成 torch 能识别的 Dataset
    torch_dataset = Data.TensorDataset(torch.FloatTensor(x_data), torch.LongTensor(y_data))

    # 把 dataset 放入 DataLoader
    loader = Data.DataLoader(
        dataset=torch_dataset,      # torch TensorDataset format
        batch_size=BATCH_SIZE,      # mini batch size
        shuffle=True,               # 要不要打乱数据 (打乱比较好)
    )
    return loader
# CNN_energy
class CNN_energy(nn.Module):
    def __init__(self):
        super(CNN_energy, self).__init__()
        # nn.Sequential()可以快速搭建神經網路
        # 卷積運算所使用的mask一般稱為kernel map，這邊為5x5的map
        # stride決定kernel map一次要走幾格
        # 上面用5x5的kernel map去跑28x28的圖片，卷積完會只剩下26x26，故加兩層
        # zero-padding 讓圖片大小相同
        self.conv1 = nn.Sequential( # input shape(channel=1, height=28, weight=28)
            nn.Conv2d(
                in_channels = 1, # 輸入信號的通道
                out_channels = 4, # 卷積產生的通道
                kernel_size = (3, 5), # 卷積核的尺寸
                stride = 1, # 卷積步長
                padding = (1,2) # 輸入的每一條邊補充0的層數
            ),  # output shape(4, 10, 16)
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            # input shape(4, 10, 16)
            nn.Conv2d(4, 8, (3,5), 1, (1,2)), # output shape(8, 10, 16)
            nn.ReLU()
        )
        self.hidden = nn.Linear(8*10*16, 800)
        self.hidden2 = nn.Linear(800, 800)
        self.hidden3 = nn.Linear(800, 800)
        self.out = nn.Linear(800, 13) # fully connected layer
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1) # 展平多維卷積圖層 (batch_size, 32*10*16)
        x = self.hidden(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        output = F.softmax(self.out(x))
        return output
# CNN_parameter
class CNN_parameter(nn.Module):
    def __init__(self):
        super(CNN_parameter, self).__init__()
        # nn.Sequential()可以快速搭建神經網路
        # 卷積運算所使用的mask一般稱為kernel map，這邊為5x5的map
        # stride決定kernel map一次要走幾格
        # 上面用5x5的kernel map去跑28x28的圖片，卷積完會只剩下26x26，故加兩層
        # zero-padding 讓圖片大小相同
        self.conv1 = nn.Sequential( # input shape(channel=1, height=28, weight=28)
            nn.Conv2d(
                in_channels = 10, # 輸入信號的通道
                out_channels = 20, # 卷積產生的通道
                kernel_size = (3, 5), # 卷積核的尺寸
                stride = 1, # 卷積步長
                padding = (1,2) # 輸入的每一條邊補充0的層數
            ),  # output shape(20, 16, 20)
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            # input shape(4, 10, 16)
            nn.Conv2d(20, 40, (3,5), 1, (1,2)), # output shape(40, 16, 20)
            nn.ReLU()
        )
        self.hidden = nn.Linear(40*16*20, 800)
        self.hidden2 = nn.Linear(800, 800)
        self.hidden3 = nn.Linear(800, 800)
        self.out = nn.Linear(800, 13) # fully connected layer
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1) # 展平多維卷積圖層 (batch_size, 32*10*16)
        x = self.hidden(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        output = F.softmax(self.out(x))
        return output
# 訓練模型
def module_training(EPOCH, LR, module_class):
    print('【訓練模型】')
    module = module_class()
    print(module)
    # optimize all cnn parameters
    optimizer = torch.optim.Adam(module.parameters(), lr=LR)
    # the target label is not one-hotted
    # pytorch 的 CrossEntropyLoss 會自動把張量轉為 one hot形式
    loss_func = nn.CrossEntropyLoss()

    # training and testing
    for epoch in range(EPOCH):
        print('第', epoch, '次訓練')
        # enumerate : 枚舉可列舉對象．ex.
        # A = [a, b, c]
        # list(enumerate(A)) = [(0,a), (1,b), (2,c)]
        for step, (b_x, b_y) in enumerate(loader):
            output = module(b_x)
            loss = loss_func(output, b_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (step % 50 == 0):
                print('step:' + str(step))
                print('loss:' + str(loss))
    print('訓練完成！！\n')
    return module

In [50]:
# 訓練結果分析
def testing_report(module, testing_x_data, testing_y_data):
    print('【訓練結果報告】')
    test_output = module(torch.FloatTensor(testing_x_data))
    pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
    # 混淆矩陣
    confusion_matrix = metrics.confusion_matrix(testing_y_data,pred_y)
    print('混淆矩陣')
    print('true/predict')
    print(confusion_matrix)
    classification_report = metrics.classification_report(testing_y_data,pred_y)
    print('\n<============================>\n\n分類報告')
    print(classification_report)

    print('<============================>\n\n分類準確率')
    correct = 0
    for i in range(len(testing_y_data)):
        if testing_y_data[i] == pred_y[i]:
            correct += 1
    print('測試資料數：', len(testing_y_data), ', 預測正確數：', correct, '準確率：', (correct/len(testing_y_data)))
    return confusion_matrix, classification_report, (correct/len(testing_y_data))

In [51]:
data = data_reading('S_All_A1_E1')
subject_emg, subject_restimulus = subject_choose(data, [4,7])
x_data, y_data = wavelet_with_energy_sum(200, 100, 'db5', 'symmetric', 4)
loader = data_setting(1024)
module = module_training(200, 0.0005, CNN_energy)

begin = 0
end = len(y_data)
testing_report(module, x_data[begin:end], y_data[begin:end])

【讀取檔案】
檔案讀取完畢！共讀取了 27 個檔案的資料

【讀取受試者資料】
受試者 4 資料讀取
受試者 7 資料讀取
受試者資料讀取完畢！

【預處理-小波包能量】
第1位受試者資料預處理
第2位受試者資料預處理
資料預處理完畢！共 2016 筆資料

資料標籤數量分布： [1127   78   81   71   71   68   80   79   75   75   81   59   71]

資料篩選完成
資料數量： x ->  965 , y -> 965
資料標籤分佈： [76 78 81 71 71 68 80 79 75 75 81 59 71] 

【資料集設定】
【訓練模型】
CNN_energy(
  (conv1): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 5), stride=(1, 1), padding=(1, 2))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(4, 8, kernel_size=(3, 5), stride=(1, 1), padding=(1, 2))
    (1): ReLU()
  )
  (hidden): Linear(in_features=1280, out_features=800, bias=True)
  (hidden2): Linear(in_features=800, out_features=800, bias=True)
  (hidden3): Linear(in_features=800, out_features=800, bias=True)
  (out): Linear(in_features=800, out_features=13, bias=True)
)
第 0 次訓練


c:\users\ericfang\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


step:0
loss:tensor(2.5649, grad_fn=<NllLossBackward>)
第 1 次訓練
step:0
loss:tensor(2.5591, grad_fn=<NllLossBackward>)
第 2 次訓練
step:0
loss:tensor(2.5482, grad_fn=<NllLossBackward>)
第 3 次訓練
step:0
loss:tensor(2.5360, grad_fn=<NllLossBackward>)
第 4 次訓練
step:0
loss:tensor(2.5252, grad_fn=<NllLossBackward>)
第 5 次訓練
step:0
loss:tensor(2.5105, grad_fn=<NllLossBackward>)
第 6 次訓練
step:0
loss:tensor(2.4986, grad_fn=<NllLossBackward>)
第 7 次訓練
step:0
loss:tensor(2.4823, grad_fn=<NllLossBackward>)
第 8 次訓練
step:0
loss:tensor(2.4666, grad_fn=<NllLossBackward>)
第 9 次訓練
step:0
loss:tensor(2.4569, grad_fn=<NllLossBackward>)
第 10 次訓練
step:0
loss:tensor(2.4475, grad_fn=<NllLossBackward>)
第 11 次訓練
step:0
loss:tensor(2.4364, grad_fn=<NllLossBackward>)
第 12 次訓練
step:0
loss:tensor(2.4240, grad_fn=<NllLossBackward>)
第 13 次訓練
step:0
loss:tensor(2.4117, grad_fn=<NllLossBackward>)
第 14 次訓練
step:0
loss:tensor(2.4000, grad_fn=<NllLossBackward>)
第 15 次訓練
step:0
loss:tensor(2.3870, grad_fn=<NllLossBackward>)
第 16 次訓練
s

step:0
loss:tensor(2.0974, grad_fn=<NllLossBackward>)
第 131 次訓練
step:0
loss:tensor(2.1076, grad_fn=<NllLossBackward>)
第 132 次訓練
step:0
loss:tensor(2.1094, grad_fn=<NllLossBackward>)
第 133 次訓練
step:0
loss:tensor(2.1154, grad_fn=<NllLossBackward>)
第 134 次訓練
step:0
loss:tensor(2.1178, grad_fn=<NllLossBackward>)
第 135 次訓練
step:0
loss:tensor(2.1121, grad_fn=<NllLossBackward>)
第 136 次訓練
step:0
loss:tensor(2.0997, grad_fn=<NllLossBackward>)
第 137 次訓練
step:0
loss:tensor(2.1064, grad_fn=<NllLossBackward>)
第 138 次訓練
step:0
loss:tensor(2.1222, grad_fn=<NllLossBackward>)
第 139 次訓練
step:0
loss:tensor(2.0942, grad_fn=<NllLossBackward>)
第 140 次訓練
step:0
loss:tensor(2.1067, grad_fn=<NllLossBackward>)
第 141 次訓練
step:0
loss:tensor(2.1029, grad_fn=<NllLossBackward>)
第 142 次訓練
step:0
loss:tensor(2.1035, grad_fn=<NllLossBackward>)
第 143 次訓練
step:0
loss:tensor(2.1059, grad_fn=<NllLossBackward>)
第 144 次訓練
step:0
loss:tensor(2.1077, grad_fn=<NllLossBackward>)
第 145 次訓練
step:0
loss:tensor(2.1092, grad_fn=<NllL

c:\users\ericfang\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.6145077720207254

In [62]:
data = data_reading('S_All_A1_E1')
# [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
subject = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
subject_emg, subject_restimulus = subject_choose(data, subject)
x_data, y_data = wavelet_parameters_only(200, 100, 'db5', 'symmetric', 4)
loader = data_setting(1024)
module = module_training(200, 0.0005, CNN_parameter)

begin = 0 
end = len(y_data)
testing_report(module, x_data[begin:end], y_data[begin:end])

【讀取檔案】
檔案讀取完畢！共讀取了 27 個檔案的資料

【讀取受試者資料】
受試者 1 資料讀取
受試者 2 資料讀取
受試者 3 資料讀取
受試者 4 資料讀取
受試者 5 資料讀取
受試者 6 資料讀取
受試者 7 資料讀取
受試者 8 資料讀取
受試者 9 資料讀取
受試者 10 資料讀取
受試者 11 資料讀取
受試者 12 資料讀取
受試者 13 資料讀取
受試者 14 資料讀取
受試者 15 資料讀取
受試者 16 資料讀取
受試者 17 資料讀取
受試者 18 資料讀取
受試者 19 資料讀取
受試者 20 資料讀取
受試者 21 資料讀取
受試者 22 資料讀取
受試者 23 資料讀取
受試者 24 資料讀取
受試者 25 資料讀取
受試者資料讀取完畢！

【預處理-小波包係數】
第1位受試者資料預處理
第2位受試者資料預處理
第3位受試者資料預處理
第4位受試者資料預處理
第5位受試者資料預處理
第6位受試者資料預處理
第7位受試者資料預處理
第8位受試者資料預處理
第9位受試者資料預處理
第10位受試者資料預處理
第11位受試者資料預處理
第12位受試者資料預處理
第13位受試者資料預處理
第14位受試者資料預處理
第15位受試者資料預處理
第16位受試者資料預處理
第17位受試者資料預處理
第18位受試者資料預處理
第19位受試者資料預處理
第20位受試者資料預處理
第21位受試者資料預處理
第22位受試者資料預處理
第23位受試者資料預處理
第24位受試者資料預處理
第25位受試者資料預處理
資料預處理完畢！共 25274 筆資料

資料標籤數量分布： [14379   951   942  1017   873   888   907   941   962   852   834   814
   914]

資料篩選完成
資料數量： x ->  11899 , y -> 11899
資料標籤分佈： [1004  951  942 1017  873  888  907  941  962  852  834  814  914] 

(11899, 10, 16, 20)
【資料集設定】
【訓練模型】
CNN_parameter(
  (conv1): Sequential(
    (0): Conv2d(10, 20, kern

c:\users\ericfang\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


step:0
loss:tensor(2.5651, grad_fn=<NllLossBackward>)
第 1 次訓練
step:0
loss:tensor(2.4492, grad_fn=<NllLossBackward>)
第 2 次訓練
step:0
loss:tensor(2.4175, grad_fn=<NllLossBackward>)
第 3 次訓練
step:0
loss:tensor(2.3640, grad_fn=<NllLossBackward>)
第 4 次訓練
step:0
loss:tensor(2.3557, grad_fn=<NllLossBackward>)
第 5 次訓練
step:0
loss:tensor(2.3608, grad_fn=<NllLossBackward>)
第 6 次訓練
step:0
loss:tensor(2.3248, grad_fn=<NllLossBackward>)
第 7 次訓練
step:0
loss:tensor(2.2941, grad_fn=<NllLossBackward>)
第 8 次訓練
step:0
loss:tensor(2.3227, grad_fn=<NllLossBackward>)
第 9 次訓練
step:0
loss:tensor(2.2824, grad_fn=<NllLossBackward>)
第 10 次訓練
step:0
loss:tensor(2.2677, grad_fn=<NllLossBackward>)
第 11 次訓練
step:0
loss:tensor(2.2537, grad_fn=<NllLossBackward>)
第 12 次訓練
step:0
loss:tensor(2.2616, grad_fn=<NllLossBackward>)
第 13 次訓練
step:0
loss:tensor(2.2503, grad_fn=<NllLossBackward>)
第 14 次訓練
step:0
loss:tensor(2.2529, grad_fn=<NllLossBackward>)
第 15 次訓練
step:0
loss:tensor(2.2272, grad_fn=<NllLossBackward>)
第 16 次訓練
s

第 130 次訓練
step:0
loss:tensor(1.9097, grad_fn=<NllLossBackward>)
第 131 次訓練
step:0
loss:tensor(1.9036, grad_fn=<NllLossBackward>)
第 132 次訓練
step:0
loss:tensor(1.9060, grad_fn=<NllLossBackward>)
第 133 次訓練
step:0
loss:tensor(1.8990, grad_fn=<NllLossBackward>)
第 134 次訓練
step:0
loss:tensor(1.8977, grad_fn=<NllLossBackward>)
第 135 次訓練
step:0
loss:tensor(1.9088, grad_fn=<NllLossBackward>)
第 136 次訓練
step:0
loss:tensor(1.9051, grad_fn=<NllLossBackward>)
第 137 次訓練
step:0
loss:tensor(1.9197, grad_fn=<NllLossBackward>)
第 138 次訓練
step:0
loss:tensor(1.9268, grad_fn=<NllLossBackward>)
第 139 次訓練
step:0
loss:tensor(1.8932, grad_fn=<NllLossBackward>)
第 140 次訓練
step:0
loss:tensor(1.9211, grad_fn=<NllLossBackward>)
第 141 次訓練
step:0
loss:tensor(1.8981, grad_fn=<NllLossBackward>)
第 142 次訓練
step:0
loss:tensor(1.9099, grad_fn=<NllLossBackward>)
第 143 次訓練
step:0
loss:tensor(1.8984, grad_fn=<NllLossBackward>)
第 144 次訓練
step:0
loss:tensor(1.9088, grad_fn=<NllLossBackward>)
第 145 次訓練
step:0
loss:tensor(1.8930, gra

c:\users\ericfang\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



<============================>

分類報告
              precision    recall  f1-score   support

           0       0.76      0.81      0.79      1004
           1       0.81      0.86      0.83       951
           2       0.84      0.95      0.89       942
           3       0.72      0.88      0.79      1017
           4       0.92      0.87      0.90       873
           5       0.86      0.85      0.85       888
           6       0.87      0.82      0.84       907
           7       0.82      0.89      0.85       941
           8       0.83      0.85      0.84       962
           9       0.80      0.77      0.78       852
          10       0.71      0.90      0.79       834
          11       0.73      0.95      0.83       814
          12       0.00      0.00      0.00       914

    accuracy                           0.80     11899
   macro avg       0.74      0.80      0.77     11899
weighted avg       0.74      0.80      0.77     11899

<============================>

分類準確率
測試資

0.7993949071350533

In [ ]:
def do_one_training(epoch, batch_size, CNN_neuron_num, sliding_window_size, sliding_window_movement, feature_extract, learning_rate, subject):
    data = data_reading('S_All_A1_E1')
    subject_emg, subject_restimulus = subject_choose(data, subject)
    x_data, y_data = wavelet_parameters_only(sliding_window_size, sliding_window_movement, 'db5', 'symmetric', 4)
    loader = data_setting(batch_size)
    module = module_training(epoch, learning_rate, CNN_parameter)

    begin = 0 
    end = len(y_data)
    testing_report(module, x_data[begin:end], y_data[begin:end])